In [ ]:
# 그래프, 수학 기능 추가
# Add graph and math features
import pylab as py
import numpy as np
import numpy.linalg as nl
# 기호 연산 기능 추가
# Add symbolic operation capability
import sympy as sy

In [ ]:
sy.init_printing()

# 룽게-쿠타법 (RK4) : 고차 상미방<br>Runge-Kutta Method (RK4) : Higher Order ODE

## 단진자<br>Simple Pendulum

다음 미분 방정식은 단진자의 운동을 묘사한다.<br>
Following differential equation describes the motion of a simple pendulum.<br>
Ref : Wikipedia contributors, 'Pendulum (mathematics)', Wikipedia, The Free Encyclopedia, 2 June 2018, 13:28 UTC, <https://en.wikipedia.org/w/index.php?title=Pendulum_(mathematics)&oldid=844080803> [accessed 5 August 2018]

$$
\frac{d^2\theta}{dt^2} + \frac{g}{l}sin\theta = 0
$$

상태변수는 다음과 같다고 가정하자.<br>
Let's assume that the state variables are as follows.

$$
\mathbf{x}
=
\begin{pmatrix}
x_0\\
x_1
\end{pmatrix}
=
\begin{pmatrix}
\theta\\
\frac{d}{dt}\theta
\end{pmatrix}
$$

상태변수의 미분은 다음과 같다.<br>Differentiation of the state variables are as follows.

$$
\frac{d}{dt}
\begin{pmatrix}
    x_0\\
    x_1
\end{pmatrix} 
=
\begin{pmatrix}
    x_1\\
    -\frac{g}{l}sinx_0
\end{pmatrix} 
$$

python 함수로 구현해 보면 다음과 같을 것이다.<br>
One possible python implementation would be as follows.

In [ ]:
g_mpsps = 9.8
l_m = 0.3

legends = ('$\\theta(deg)$', '$\\frac{d}{dt}\\theta(deg/s)$')
ylabel = ''

# Initial state
x_0 = np.array([np.deg2rad(90), 0])

def pendulum_NL(x, t):
    """
    Parameters
    ==========
    x: array of theta and d(theta)/dt
    t: time value
    
    Return Value
    ============
    One dimensional array of dx/dt
    """
    
    return np.array([x[1], (-g_mpsps/l_m)*np.sin(x[0])])


룽게-쿠타법을 오일러법, 훈법과 비교해보자.<br>Let's compare the Runge-Kutta method with Euler method, and Heun's method.

In [ ]:
def rk4_step(f, x0, t0, t1):
    """
    One time step of Runge-Kutta method

    f: dx_dt function
    x0 : initial condition
    t0 : this step time
    t1 : next step time
    """
    delta_t = (t1 - t0)
    delta_t_half = delta_t * 0.5
    t_half = t0 + delta_t_half
    
    # Step 1
    s1 = f(x0, t0)

    # Step 2
    s2 = f(x0 + s1 * delta_t_half, t_half)

    # Step 3
    s3 = f(x0 + s2 * delta_t_half, t_half)

    # Step 4
    s4 = f(x0 + s3 * delta_t, t1)

    # Step 5
    s = (1.0 / 6.0) * (s1 + (s2 + s3) * 2 + s4)

    # Step 6
    x1 = x0 + s * delta_t

    return x1


In [ ]:
def rk4(f, t_array, x_0):
    time_list = [t_array[0]]
    result_list = [x_0]

    x_i = x_0

    for k, t_i in enumerate(t_array[:-1]):
        # time step
        x_i_plus_1 = rk4_step(f, x_i, t_i, t_array[k+1])

        time_list.append(t_array[k+1])
        result_list.append(x_i_plus_1)
        
        x_i = x_i_plus_1

    return time_list, result_list


In [ ]:
def euler(f, t_array, x_0):
    time_list = [t_array[0]]
    result_list = [x_0]

    x_i = x_0

    for k, t_i in enumerate(t_array[:-1]):
        # time step
        delta_t = t_array[k+1] - t_array[k]

        # slope
        s_i = f(x_i, t_i)

        # x[i + 1]
        x_i_plus_1 = x_i + s_i * delta_t

        time_list.append(t_array[k+1])
        result_list.append(x_i_plus_1)
        
        x_i = x_i_plus_1

    return time_list, result_list


In [ ]:
def heun(f, t_array, x_0):
    time_list = [t_array[0]]
    result_list = [x_0]

    x_i = x_0

    for k, t_i in enumerate(t_array[:-1]):
        # time step
        delta_t = t_array[k+1] - t_array[k]

        # slope at i
        s_i = f(x_i, t_i)

        # x[i + 1] by Forward Euler
        x_i_plus_1 = x_i + s_i * delta_t
        
        # slope at i + 1
        s_i_plus_1 = f(x_i_plus_1, t_array[k+1])
        
        # average of slope
        s_average = (s_i + s_i_plus_1) * 0.5
        
        # x[i + 1] by Modified Euler
        x_i_plus_1_m = x_i + s_average * delta_t

        time_list.append(t_array[k+1])
        result_list.append(x_i_plus_1_m)
        
        x_i = x_i_plus_1_m

    return time_list, result_list


In [ ]:
# Time array
delta_t = 0.001

t_sec_array = np.arange(0, 6 + delta_t*0.5, delta_t)

# *** Euler ***
t_euler, x_euler = euler(pendulum_NL, t_sec_array, x_0)
py.plot(t_euler, x_euler, '-', label='Euler')

# *** Heun ***
t_heun, x_heun = heun(pendulum_NL, t_sec_array, x_0)
py.plot(t_heun, x_heun, '-', label='Heun')

# *** RK4 ***
t_rk4, x_rk4 = rk4(pendulum_NL, t_sec_array, x_0)
py.plot(t_rk4, x_rk4, '-', label='RK4')

py.xlabel('t(sec)')
py.ylabel('x(m)')

py.legend(loc=0)
py.grid(True)
